In [ ]:
import selenium
from selenium import webdriver

In [ ]:
import time
# 相簿效期只有3個月，須判斷是否仍在效期內。（上傳日期：2018-07-29）

localtime = time.asctime( time.localtime(time.time()) )
# time.time() : seconds since 12:00am, January 1, 1970
# time.localtime() : show localtime in struct_time
# time.asctime() : show time in "Www Mmm dd hh:mm:ss yyyy" formate

dateStruct_time = time.strptime("2018-10-28 23:59:59", "%Y-%m-%d %H:%M:%S")
urlInvalidDate = time.asctime(dateStruct_time)

In [ ]:
driver = webdriver.Chrome(r'D:\pyLadies\2018_0805_nmnsCrawler\chromedriver.exe')

urlDay1 = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13'
urlDay2 = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=14'
urlDay3 = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=15'

driver.get(urlDay1)

In [ ]:
# 圖片網址可能所在之 xpath & attribute 對應
xpath_attribute = {
    '//div[@data-swiper-slide-index]':'style',
    '//div[@data-swiper-slide-index]/img':'src'
}

In [ ]:
# method : get the clean URLs
def findURL(xpath,attribute):
    images = driver.find_elements_by_xpath(xpath)
    index = 0 #圖片張數計算

    for image in images:
        tagContent = image.get_attribute(attribute)
        
        if tagContent.find('\"') != -1 : tagContent = tagContent.split('\"')[-1]

        if tagContent.find("https://") != -1 :
            index += 1
            url = tagContent.split('\"')[0]
            print(url, "no.",index)        
        else:
            return False

In [ ]:
# main
if localtime < urlInvalidDate :
    print("相簿已逾期！請手動確認檔案是否仍存在。")
else : # 找網址    
    for xpath,attribute in  xpath_attribute.items():
        if findURL(xpath,attribute) != False:
            print("取得網址！")
        else :
            print( "注意：「", xpath, " + ", attribute, "」的組合失敗。" )

In [ ]:
driver.close()

url = "http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13" 

ctrl + /
